# Get Images From Video

개별 비디오에서 이미지를 한개씩 수집한다. 이미지의 개수는 100개씩으로, climb와 not climb로 구성된다.

In [19]:
#set location to dataset folder
#cd Dataset/climb/
#!dir

In [29]:
# install (run only once at local)
import sys
print(sys.executable)
import sys
!{sys.executable} -m pip install yt-dlp
!{sys.executable} -m pip install pytube
!{sys.executable} -m pip install moviepy

try:
    import yt_dlp
    from pytube import YouTube
    from moviepy.editor import VideoFileClip
    print("All imports successful!")
except ImportError as e:
    print(f"Import failed: {e}")


C:\Users\USER\anaconda3\python.exe
  Using cached yt_dlp-2024.4.9-py3-none-any.whl.metadata (165 kB)
  Using cached mutagen-1.47.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached pycryptodomex-3.20.0-cp35-abi3-win_amd64.whl.metadata (3.4 kB)
Using cached yt_dlp-2024.4.9-py3-none-any.whl (3.1 MB)
   ---------------------------------------- 0.0/125.0 kB ? eta -:--:--
   -------------------------- ------------- 81.9/125.0 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------  122.9/125.0 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 125.0/125.0 kB 1.1 MB/s eta 0:00:00
Using cached mutagen-1.47.0-py3-none-any.whl (194 kB)
Using cached pycryptodomex-3.20.0-cp35-abi3-win_amd64.whl (1.8 MB)
  Using cached pytube-15.0.0-py3-none-any.whl.metadata (5.0 kB)
Using cached pytube-15.0.0-py3-none-any.whl (57 kB)
     ---------------------------------------- 0.0/388.3 kB ? eta -:--:--
     ------ -------------------------------- 61.4/388.3 kB 1.7 MB/s eta 0:00:01
 

In [31]:
# install (run only once at local)
import sys
!{sys.executable} -m pip install opencv-python
import cv2
print(cv2.__version__)


  Using cached opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl (38.6 MB)
4.9.0


## import

In [1]:
import numpy as np
import pandas as pd
import yt_dlp
from pytube import YouTube
from datetime import datetime
from moviepy.editor import VideoFileClip
import cv2
import os
import shutil

# climb image

이미 수집된 image metadata정보를 이용해서 동일 출처 비디오의 경우 중간 인덱스의 이미지만 남긴다.

In [39]:
# metadata로드
climb_metadata = pd.read_csv('./climb_metadata_set.csv', encoding='cp1252')
climb_metadata

,Author,Title,URL,search_keyword,duration_sec,video_num,itag,mime_tyoe,resolution,fps,video_codec,audio_codec,progressive,type,start_min,start_sec,finish_min,finish_sec,data_num
0,Hannah Morris Bouldering,I Trained Like A Professional Climber for 40 DAYS,https://www.youtube.com//watch?v=txGhHkbPjCk&p...,climbing training,960,0,22,video/mp4,720p,24,avc1.64001F,mp4a.40.2,True,video,14,57,14,59,1
1,HoseokClimb,My Rock Climbing Training Routine to V8 Reveal...,https://www.youtube.com//watch?v=IHHEKcfVIsg&p...,climbing training,1434,1,22,video/mp4,720p,30,avc1.64001F,mp4a.40.2,True,video,21,20,21,23,2
2,Hooper's Beta,How to Train for Climbing [Full Body Guide ft....,https://www.youtube.com//watch?v=Iy8wi_u5oFI&p...,climbing training,4149,2,22,video/mp4,720p,30,avc1.64001F,mp4a.40.2,True,video,66,0,66,2,3
3,Adam Ondra,Climb Smart: Boulder Training Efficiency ? | T...,https://www.youtube.com//watch?v=SLJTwFQE10A&p...,climbing training,1402,3,22,video/mp4,720p,25,avc1.64001F,mp4a.40.2,True,video,17,12,17,18,4
4,Adam Ondra,Most Climbers Should Train LIKE THIS | Adam On...,https://www.youtube.com//watch?v=xCu89k1aGWo&p...,climbing training,1708,4,22,video/mp4,720p,24,avc1.64001F,mp4a.40.2,True,video,4,50,4,54,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,Climbing World,?? Tomoa Narasaki OUTDOOR ROCK CLIMBING - [V11...,https://www.youtube.com//watch?v=EdZ_HK4rS58&p...,climbing v11,736,247,22,video/mp4,720p,30,avc1.64001F,mp4a.40.2,True,video,5,21,5,24,246
233,Sam Rothstein,Dreamland V11,https://www.youtube.com//watch?v=G9jW-L8zxAk&p...,climbing v11,50,252,22,video/mp4,720p,30,avc1.64001F,mp4a.40.2,True,video,0,9,0,11,247
234,Sam Rothstein,Dreamland V11,https://www.youtube.com//watch?v=G9jW-L8zxAk&p...,climbing v11,50,252,22,video/mp4,720p,30,avc1.64001F,mp4a.40.2,True,video,0,40,0,45,248
235,Tension Climbing,"Roy Quanstrom sends ""King Tut"" V11 in Red Feather",https://www.youtube.com//watch?v=kSWIWgTvM-4&p...,climbing v11,75,255,22,video/mp4,720p,24,avc1.64001F,mp4a.40.2,True,video,1,2,1,6,249


In [57]:
# 선택한 중간 이미지 이외의 행을 제거
climb_metadata_unique = pd.concat([climb_metadata[~climb_metadata.duplicated(subset='URL', keep=False)], middle_images])
climb_metadata_unique.shape


(149, 19)

In [53]:
# 파일을 이동한다Dataset/climb
os.makedirs("../../Dataset/mg_climb", exist_ok=True)

for data in climb_metadata_unique['data_num']:
    src_path = f"../../Dataset/climb/datanum_{data}.jpg"
    dst_path = f"../../Dataset/mg_climb/datanum_{data}.jpg"
    if os.path.exists(src_path):
        shutil.move(src_path, dst_path)

In [56]:
# "mg_climb" 폴더 안의 파일 리스트를 가져온다
file_list = os.listdir("../../Dataset/mg_climb")
# 파일 리스트 길이 출력
print(len(file_list))

148


In [58]:
# 직접 데이터 확인 후 이상한 이미지 삭제
# climb_metadata_unique에서 data_num이 3, 33, 156인 행을 필터링하여 삭제합니다.
climb_metadata_unique = climb_metadata_unique[~climb_metadata_unique['data_num'].isin([3, 33, 156])]

# climb_metadata_set.csv
climb_metadata_unique.to_csv('./climb_metadata_mg.csv', index=False)

# 해당 이미지 파일을 삭제
for data in [3, 33, 156]:
    file_path = f"../../Dataset/mg_climb/datanum_{data}.jpg"
    if os.path.exists(file_path):
        os.remove(file_path)


# Not Climb image

수집된 image metadata 정보를 이용해서 동일 출처 비디오의 경우 중간 인덱스의 이미지로 재추출 한다.

In [6]:
not_climb_metadata = pd.read_csv('./not_climb_metadata_set.csv', encoding='cp1252')
# 제거할 특성들
features_to_remove = ['itag']
# not_climb_metadata에서 특정 특성들을 제거
not_climb_metadata= not_climb_metadata.drop(columns=features_to_remove)
not_climb_metadata= not_climb_metadata.groupby('video_num').first().reset_index()
# 결과 확인
not_climb_metadata

,video_num,Author,Title,URL,search_keyword,lable,duration_sec,start_min,start_sec,finish_min,finish_sec,data_num
0,1,Global Triathlon Network,What Is Perfect Running Form? | Run Technique ...,https://www.youtube.com//watch?v=brFHyOtTwH4&p...,jogging posture,jogging,420,4.0,16.0,4.0,17.0,1.0
1,2,Global Triathlon Network,How To Run Properly | Running Technique Explained,https://www.youtube.com//watch?v=_kGESn8ArrU&p...,jogging posture,jogging,574,1.0,17.0,1.0,18.0,4.0
2,3,Global Triathlon Network,The Perfect Running Technique? | How To Master...,https://www.youtube.com//watch?v=lWWZQpwKjhs&p...,jogging posture,jogging,1126,15.0,58.0,15.0,59.0,7.0
3,4,James Dunne,PERFECT RUNNING FORM - 5 Tips for Running Fast...,https://www.youtube.com//watch?v=sScNDZu2MWk&p...,jogging posture,jogging,382,0.0,17.0,0.0,18.0,10.0
4,6,The Barefoot Sprinter,The Difference Between Jogging & Running,https://www.youtube.com//shorts/qUMWZnt14KeY,jogging posture,jogging,45,0.0,5.0,0.0,6.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...
115,156,D Wave,How to get cut from the basketball team??????,https://www.youtube.com//shorts/PBPni0ogegw,basketball posture,basketball,40,NaN,NaN,NaN,NaN,NaN
116,157,SIKANA English,The Basic Rules of Basketball | Basketball,https://www.youtube.com//watch?v=XbtmGKif7Ck&p...,basketball posture,basketball,145,NaN,NaN,NaN,NaN,NaN
117,158,Milad Mirg,I LEARNED HOW TO SHOOT A BASKETBALL WITH PERFE...,https://www.youtube.com//shorts/NtfhmH9pTuk,basketball posture,basketball,27,NaN,NaN,NaN,NaN,NaN
118,159,ILoveBasketballTV,Get PERFECT Basketball Shooting Form ?? | Shoo...,https://www.youtube.com//watch?v=ApveFDMS7FM&p...,basketball posture,basketball,421,NaN,NaN,NaN,NaN,NaN


In [7]:
# 선택한 중간 이미지 이외의 행을 제거
not_climb_metadata_unique = pd.concat([not_climb_metadata[~not_climb_metadata.duplicated(subset='video_num', keep=False)]])
not_climb_metadata_unique['data_num'] = not_climb_metadata_unique.index.values.astype(int)
not_climb_metadata_unique

,video_num,Author,Title,URL,search_keyword,lable,duration_sec,start_min,start_sec,finish_min,finish_sec,data_num
0,1,Global Triathlon Network,What Is Perfect Running Form? | Run Technique ...,https://www.youtube.com//watch?v=brFHyOtTwH4&p...,jogging posture,jogging,420,4.0,16.0,4.0,17.0,0
1,2,Global Triathlon Network,How To Run Properly | Running Technique Explained,https://www.youtube.com//watch?v=_kGESn8ArrU&p...,jogging posture,jogging,574,1.0,17.0,1.0,18.0,1
2,3,Global Triathlon Network,The Perfect Running Technique? | How To Master...,https://www.youtube.com//watch?v=lWWZQpwKjhs&p...,jogging posture,jogging,1126,15.0,58.0,15.0,59.0,2
3,4,James Dunne,PERFECT RUNNING FORM - 5 Tips for Running Fast...,https://www.youtube.com//watch?v=sScNDZu2MWk&p...,jogging posture,jogging,382,0.0,17.0,0.0,18.0,3
4,6,The Barefoot Sprinter,The Difference Between Jogging & Running,https://www.youtube.com//shorts/qUMWZnt14KeY,jogging posture,jogging,45,0.0,5.0,0.0,6.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...
115,156,D Wave,How to get cut from the basketball team??????,https://www.youtube.com//shorts/PBPni0ogegw,basketball posture,basketball,40,NaN,NaN,NaN,NaN,115
116,157,SIKANA English,The Basic Rules of Basketball | Basketball,https://www.youtube.com//watch?v=XbtmGKif7Ck&p...,basketball posture,basketball,145,NaN,NaN,NaN,NaN,116
117,158,Milad Mirg,I LEARNED HOW TO SHOOT A BASKETBALL WITH PERFE...,https://www.youtube.com//shorts/NtfhmH9pTuk,basketball posture,basketball,27,NaN,NaN,NaN,NaN,117
118,159,ILoveBasketballTV,Get PERFECT Basketball Shooting Form ?? | Shoo...,https://www.youtube.com//watch?v=ApveFDMS7FM&p...,basketball posture,basketball,421,NaN,NaN,NaN,NaN,118


In [13]:
import os
import subprocess
import cv2
import math

# 비디오 다운로드
def download_video(url, output_dir, data_num):
    command = ["yt-dlp", "-o", os.path.join(output_dir, f"datanum_{data_num}.mp4"), url]
    subprocess.run(command)

# 비디오 파일 삭제
def delete_video(video_path):
    os.remove(video_path)

# 중간 시간에 해당하는 이미지 추출
def extract_middle_image(video_path, output_dir, start_min, start_sec, finish_min, finish_sec, data_num):
    # 시간이 nan인 경우
    if any(math.isnan(t) for t in [start_min, start_sec, finish_min, finish_sec]):
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print("Error: Could not open video file.")
            return

        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        middle_frame = total_frames // 2

        cap.set(cv2.CAP_PROP_POS_FRAMES, middle_frame)
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            cap.release()
            return

        file_name = f"datanum_{data_num}_not.jpg"
        file_path = os.path.join(output_dir, file_name)
        cv2.imwrite(file_path, frame)

        cap.release()
        print(f"이미지 저장 완료: {file_path}")
    else:
        total_seconds = (finish_min * 60 + finish_sec + start_min * 60 + start_sec)
        middle_seconds = int(total_seconds) // 2
        middle_time_str = f"{middle_seconds // 3600:02}:{middle_seconds % 3600 // 60:02}:{middle_seconds % 60:02}"

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print("Error: Could not open video file.")
            return

        cap.set(cv2.CAP_PROP_POS_MSEC, middle_seconds * 1000)
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            cap.release()
            return

        file_name = f"datanum_{data_num}_not.jpg"
        file_path = os.path.join(output_dir, file_name)
        cv2.imwrite(file_path, frame)

        cap.release()
        print(f"이미지 저장 완료: {file_path}")

# 비디오 다운로드 및 중간 이미지 추출 후 비디오 삭제
def process_video(url, output_dir, start_min, start_sec, finish_min, finish_sec, data_num):
    print("다운로드중 ...", end='')
    download_video(url, output_dir, data_num)
    print("    완료")
    video_title = f"datanum_{data_num}.mp4"
    video_path = os.path.join(output_dir, video_title)
    print(f"비디오 path : {video_path}")
    print("extract...", end='')
    extract_middle_image(video_path, output_dir, start_min, start_sec, finish_min, finish_sec, data_num)
    delete_video(video_path)
    print(f"DELETE 비디오: {video_path}")


In [17]:
for index, row in not_climb_metadata_unique[20:].iterrows():
    data_num = row['data_num']


    try:
        start_min = row['start_min']
        start_sec = row['start_sec']
        finish_min =row['finish_min']
        finish_sec =row['finish_sec']
        process_video(row['URL'], "../../Dataset/mg_not_climb", start_min, start_sec, finish_min, finish_sec, data_num)
    except Exception as e:
        print(f"An error occurred for data_num {data_num}: {e}")
        continue

다운로드중 ...    완료
비디오 path : ../../Dataset/mg_not_climb\datanum_20.mp4
extract...이미지 저장 완료: ../../Dataset/mg_not_climb\datanum_20_not.jpg
DELETE 비디오: ../../Dataset/mg_not_climb\datanum_20.mp4
다운로드중 ...    완료
비디오 path : ../../Dataset/mg_not_climb\datanum_21.mp4
extract...이미지 저장 완료: ../../Dataset/mg_not_climb\datanum_21_not.jpg
DELETE 비디오: ../../Dataset/mg_not_climb\datanum_21.mp4
다운로드중 ...    완료
비디오 path : ../../Dataset/mg_not_climb\datanum_22.mp4
extract...이미지 저장 완료: ../../Dataset/mg_not_climb\datanum_22_not.jpg
DELETE 비디오: ../../Dataset/mg_not_climb\datanum_22.mp4
다운로드중 ...    완료
비디오 path : ../../Dataset/mg_not_climb\datanum_23.mp4
extract...이미지 저장 완료: ../../Dataset/mg_not_climb\datanum_23_not.jpg
DELETE 비디오: ../../Dataset/mg_not_climb\datanum_23.mp4
다운로드중 ...    완료
비디오 path : ../../Dataset/mg_not_climb\datanum_24.mp4
extract...이미지 저장 완료: ../../Dataset/mg_not_climb\datanum_24_not.jpg
DELETE 비디오: ../../Dataset/mg_not_climb\datanum_24.mp4
다운로드중 ...    완료
비디오 path : ../../Dataset/mg_not_cl